# **IMDB_Movie_Review_Sentiment_Analysis**

In [65]:
!pip install kaggle


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other




  0%|          | 0.00/25.7M [00:00<?, ?B/s]
  4%|▍         | 1.00M/25.7M [00:02<01:08, 380kB/s]
  8%|▊         | 2.00M/25.7M [00:04<00:50, 494kB/s]
 12%|█▏        | 3.00M/25.7M [00:07<00:56, 425kB/s]
 16%|█▌        | 4.00M/25.7M [00:08<00:45, 501kB/s]
 19%|█▉        | 5.00M/25.7M [00:10<00:40, 535kB/s]
 23%|██▎       | 6.00M/25.7M [00:14<00:49, 421kB/s]
 27%|██▋       | 7.00M/25.7M [00:16<00:44, 443kB/s]
 31%|███       | 8.00M/25.7M [00:17<00:36, 503kB/s]
 35%|███▌      | 9.00M/25.7M [00:19<00:32, 535kB/s]
 39%|███▉      | 10.0M/25.7M [00:20<00:29, 566kB/s]
 43%|████▎     | 11.0M/25.7M [00:22<00:24, 643kB/s]
 47%|████▋     | 12.0M/25.7M [00:23<00:20, 703kB/s]
 51%|█████     | 13.0M/25.7M [00:24<00:17, 770kB/s]
 54%|█████▍    | 14.0M/25.7M [00:26<00:17, 719kB/s]
 58%|█████▊    | 15.0M/25.7M [00:28<00:17, 637kB/s]
 62%|██████▏   | 16.0M/25.7M [00:29<00:14, 726kB/s]
 66%|██████▌   | 17.0M/25.7M [00:30<00:11, 803kB/s]
 70%|███████   | 18.0M/25.7M [00:31<00:09, 854kB/s]
 74%|███████▍  | 19

In [67]:
# Extracting the compressed dataset
from zipfile import ZipFile

# Path to the dataset
dataset = r'D:\work\imdb-dataset-of-50k-movie-reviews.zip'  # Use raw string (r'') for Windows paths to avoid escaping issues

# Extracting the dataset
with ZipFile(dataset, 'r') as zip_file:
    zip_file.extractall()
    print('The dataset has been successfully extracted.')

The dataset has been successfully extracted.


In [68]:
# import librarys

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout

In [69]:
# Load dataset
data = pd.read_csv("IMDB Dataset.csv")

In [70]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
# Get the shape of the data
data.shape

(50000, 2)

In [43]:
# Convert sentiment labels to binary (positive=1, negative=0)
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [44]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
# Count the number of occurrences of each sentiment in the "sentiment" column of the data DataFrame
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [46]:
# Text cleaning function
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Arunk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
def clean_text(text):
    text = re.sub(r"<.*?>", "", text)  # Remove HTML tags
    text = re.sub(r"[^a-zA-Z]", " ", text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = " ".join(word for word in text.split() if word not in stop_words)  # Remove stopwords
    return text

  Cleans the input text by removing HTML tags, special characters, converting to lowercase, and filtering out stopwords.

In [48]:
# Apply text cleaning
data["review"] = data["review"].apply(clean_text)

In [ ]:
data.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ho...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter mattei love time money visually stunnin...,1


In [50]:
# Splitting dataset
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Count the number of occurrences of each sentiment in the "sentiment" column of the data DataFrame
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [52]:
# Tokenization
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_data["review"])

In [53]:
# Convert to sequences & pad them
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [54]:
# Labels
Y_train = np.array(train_data["sentiment"])
Y_test = np.array(test_data["sentiment"])

In [55]:
# Model creation
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=200))
model.add(Conv1D(64, kernel_size=5, activation="relu"))  # CNN Layer
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))  # BiLSTM Layer
model.add(Dense(1, activation="sigmoid"))  # Output layer

In [56]:
# Compile model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [57]:
# Model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [58]:
# Train model
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 267s 520ms/step - accuracy: 0.7713 - loss: 0.4483 - val_accuracy: 0.8754 - val_loss: 0.3002
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 313s 626ms/step - accuracy: 0.9181 - loss: 0.2199 - val_accuracy: 0.8854 - val_loss: 0.2925
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 286s 572ms/step - accuracy: 0.9483 - loss: 0.1443 - val_accuracy: 0.8851 - val_loss: 0.3220
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 276s 552ms/step - accuracy: 0.9692 - loss: 0.0961 - val_accuracy: 0.8720 - val_loss: 0.3743
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 296s 592ms/step - accuracy: 0.9808 - loss: 0.0625 - val_accuracy: 0.8701 - val_loss: 0.4678


In [59]:
import pickle
import joblib
from tensorflow.keras.models import load_model

# Save the trained model
model.save("sentiment_model.h5")  # Save in HDF5 format

# Save the tokenizer using pickle
with open("tokenizer.pkl", "wb") as file:
    pickle.dump(tokenizer, file)

print("✅ Model and Tokenizer saved successfully!")

✅ Model and Tokenizer saved successfully!


In [60]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, Y_test)
# Print the accuracy of the model on the test data
print(f"Test Accuracy: {accuracy * 100:.2f}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 25s 76ms/step - accuracy: 0.8705 - loss: 0.4577
Test Accuracy: 87.09%


In [61]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict on test data
Y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Confusion Matrix
print("Confusion Matrix:\n", confusion_matrix(Y_test, Y_pred))

# Precision, Recall, F1-score
print("Classification Report:\n", classification_report(Y_test, Y_pred))

313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step
Confusion Matrix:
 [[4430  531]
 [ 760 4279]]
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87      4961
           1       0.89      0.85      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



# predictive system

In [62]:
# Define a function called predictive_system that takes in a review as an argument
def predictive_system(review):
    sequences = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequences, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"
    return sentiment

In [63]:
predictive_system("The movie was boring and too slow.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step


'Negative'

In [64]:
predictive_system("This movie was absolutely fantastic!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


'Positive'